In [1]:
import os

import hydra
import pandas as pd
from pipeline import get_preprocessing_pipeline


In [2]:
get_preprocessing_pipeline()

{'preprocessing_pipeline': {'_target_': 'pipeline.make_pipeline', 'steps_config': [{'SimpleImputer': {'_target_': 'sklearn.impute.SimpleImputer', 'strategy': 'constant', 'fill_value': '-1'}}, {'VarianceThreshold': {'_target_': 'sklearn.feature_selection.VarianceThreshold', 'threshold': 0.1}}, {'StandardScaler': {'_target_': 'sklearn.preprocessing.StandardScaler'}}]}}


Pipeline(steps=[('SimpleImputer',
                 SimpleImputer(fill_value='-1', strategy='constant')),
                ('VarianceThreshold', VarianceThreshold(threshold=0.1)),
                ('StandardScaler', StandardScaler())])

In [3]:
accepted = pd.read_csv('data/archive/accepted.csv')
rejected = pd.read_csv('data/archive/rejected.csv')

/var/folders/5f/tg63xq8x6jl7df71n9v3_g9c0000gn/T/ipykernel_56140/1286688764.py:1: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  accepted = pd.read_csv('data/archive/accepted.csv')


KeyboardInterrupt: 